In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import gstlearn as gl
import gstlearn.plot as gp
from bramm_data_analysis import loaders
import matplotlib.pyplot as plt
from bramm_data_analysis.spatial import RegularGrid

# Données

### Chargement

On charge les données pour une variable, ici le cuivre. Les données de mousse correspondent aux données des 3 feuilles de données, fusionnées sur le code de site et sur le code d'échantillon pour former un grand jeu de données. Les données sont ensuite filtrées pour ne conserver que la variable interessante (ici le cuivre).

La gestion des valeurs dupliquées se fait en deux étapes. Dans un premier temps, on aggrège les données correspondants à un même échantillon (les données ayant même date, même longitude et même latitude), en faisant leur moyenne. Ensuite, si il reste des données ayant la même position spatial (il n'est plus question de date ici), on conserve les données les plus récentes.

On s'assure ensuite de l'absence de `NaN` dans les données et on convertit le jeu de données (on conservant seulement la longitude, la latitute et les valeurs de cuivre) au format `Db` de `gstlearn` pour effectuer un premier krigeage.

Pour plus d'informations sur le chargement des données, se référer [aux fonctions de chargement de données](../src/bramm_data_analysis/loaders/core.py).

In [ ]:
x1 = "longitude"
x2 = "latitude"
z = "copper"

# Moss
moss_data_path = Path("../data/Mines_2024.xlsx")
observations = loaders.from_moss_csv(moss_data_path).retrieve_db(
    xs=["longitude", "latitude"],
    zs=z,
    duplicates_handling_strategy="mean",
)
observations.plot(z)

### Définition d'une grille pour le krigeage

On définit une grille régulière, de maille `step` à l'aide de `gstlearn.Dbgrid.create`. Ensuite, à l'aide de la library `geopandas` et du tracé de la [france métropolitaine](../data/metropole.json), on créé un selecteur dans la grid régulière pour indiquer les points à l'intérieur de la france métropolitaine et de la corse. Pour plus de détailler, se référer au module [`spatial`](../src/bramm_data_analysis/spatial/__init__.py).

In [ ]:
step = 0.1
grid = RegularGrid.from_boundary_path(Path("../data/metropole.json"))
db_grid = grid.retrieve_grid(step)
db_grid.display()

# Variogramme

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(
    ndir=2, npas=10, dpas=0.5, toldis=0.2
)
vario2dir = gl.Vario(varioParamMulti)
err = vario2dir.compute(observations)
ax = gp.varmod(vario2dir)

In [ ]:
fitmod = gl.Model()
types = [gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.GAUSSIAN]
err = fitmod.fit(vario2dir, types=types)
ax = gp.varmod(vario2dir, fitmod)

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=db_grid,
    model=fitmod,
    neigh=gl.NeighUnique.create(),
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("SK"),
)
# visualization
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"SK.{z}.estim", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(
    title=f"Simple Kriging over whole Grid: Mean={round(observations[z].mean(),1)}"
)
plt.show()
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"SK.{z}.stdev", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(title="Simple Kriging std-dev over whole Grid")
plt.show()

In [ ]:
fitmodSK = fitmod.clone()
err = fitmodSK.setMean(mean=observations[z].mean())

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=db_grid,
    model=fitmodSK,
    neigh=gl.NeighUnique.create(),
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("Mean_SK"),
)

In [ ]:
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"Mean_SK.{z}.estim", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(
    title=f"Simple Kriging over whole Grid: Mean={round(observations[z].mean(),1)}"
)
plt.show()

In [ ]:
fitmodOK = fitmod.clone()
err = fitmodOK.addDrift(gl.DriftM())

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=db_grid,
    model=fitmodOK,
    neigh=gl.NeighUnique.create(),
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("OK"),
)

In [ ]:
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"OK.{z}.estim", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(title=f"Ordinary Kriging over whole Grid")
plt.show()
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"OK.{z}.stdev", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(title="Ordinary Kriging std-dev over whole Grid")
plt.show()

In [ ]:
ax = gp.correlation(
    db_grid,
    namex=f"OK.{z}.estim",
    namey=f"SK.{z}.estim",
    bissLine=True,
    bins=100,
    flagSameAxes=True,
    cmin=1,
)
ax.decoration(
    title="Estimation Simple vs. Ordinary",
    xlabel="Ordinary Kriging",
    ylabel="Simple Kriging",
)

In [ ]:
ax = gp.correlation(
    db_grid,
    namex=f"OK.{z}.stdev",
    namey=f"SK.{z}.stdev",
    bissLine=True,
    bins=100,
    flagSameAxes=True,
    cmin=1,
)
ax.decoration(
    title="Estimation Simple vs. Ordinary",
    xlabel="Ordinary Kriging",
    ylabel="Simple Kriging",
)